### transform tif images to JPG images

In [ ]:
import os
from PIL import Image

def convert_all_tif_to_jpg(folder_path):
    if not os.path.exists(folder_path):
        print(f"folder {folder_path} not exists")
        return
    output_folder = os.path.join(folder_path, "test_jpg")
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            tif_path = os.path.join(folder_path, filename)
            tif_image = Image.open(tif_path).convert('L')

            rgb_image = tif_image.convert('RGB')
    
            jpg_filename = os.path.splitext(filename)[0] + ".jpg"
            jpg_path = os.path.join(output_folder, jpg_filename)
            rgb_image.save(jpg_path, 'JPEG')
            print(f"changed {filename} to {jpg_filename}")
        else:
            print(f"skipped {filename} because it is not a tif file")
    
    print("all done!")

folder_path = 'DIC_crack_dataset/test'
convert_all_tif_to_jpg(folder_path)


In [ ]:
from torchvision.io.image import read_image
import matplotlib.pyplot as plt

img = read_image("SVS_2_LS18_to_LS19_RS2_0000_1_173_2816_2304.jpg")
# show the img
plt.imshow(img.permute(1, 2, 0)); plt.axis('off'); plt.tight_layout(); plt.show()

In [ ]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18
from torchcam.methods import SmoothGradCAMpp,GradCAM , GradCAMpp, ScoreCAM,LayerCAM, SSCAM,GradCAMpp, ISCAM, XGradCAM
import cv2 as cv
import torch
from torch import nn
import os


for img_path in os.listdir('test_img'):
    img = read_image(os.path.join('test_img', img_path))
    # Preprocess it for your chosen model
    input_tensor = normalize(resize(img, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    model = resnet18(pretrained=True)
    model.fc = nn.Linear(512, 2)
    model.load_state_dict(torch.load('checkpoints/output/epoch=99-val_loss=0.00.ckpt'))
    model.eval()  # Set the model to evaluation mode after loading

    cam_extractor = ScoreCAM(model, 'layer4')
    out=(model(input_tensor.unsqueeze(0)))
    activation_map=(cam_extractor(out.squeeze(0).argmax().item(), out))

    import matplotlib.pyplot as plt
    from torchcam.utils import overlay_mask

    # Resize the CAM and overlay it
    result = overlay_mask(to_pil_image(img), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.999)
    # Display it
    plt.imshow(result); plt.axis('off'); plt.tight_layout(); plt.show()


In [ ]:
import torch
# load the checkpoint
ckpt = torch.load('checkpoints/output/epoch=99-val_loss=0.00.ckpt')
# remove "model." from the state_dict keys
ckpt = ckpt['state_dict']
ckpt = {k.replace("model.", ""): v for k, v in ckpt.items()} # remove "model." from the state_dict keys
# save 
torch.save(ckpt, 'checkpoints/output/epoch=99-val_loss=0.00.ckpt')


In [ ]:
import torch
import random
import cv2 as cv
import numpy as np
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchvision.models import resnet18
from torchcam.methods import CAM, GradCAM, ScoreCAM,LayerCAM, SSCAM,GradCAMpp, ISCAM, XGradCAM, SmoothGradCAMpp
from torchcam.utils import overlay_mask


from segment_anything import sam_model_registry, SamPredictor
from torchvision.transforms.functional import to_pil_image
from torch import nn

import matplotlib.pyplot as plt

# Step 0: seed
def set_seed(seed):
    # Python built-in random seed
    random.seed(seed)
    
    # NumPy random seed
    np.random.seed(seed)
    
    # PyTorch random seed (for CPU and GPU)
    torch.manual_seed(seed)
    
    # If you are using GPU and want deterministic behavior
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # if using multi-GPU

    # Ensuring deterministic behavior in PyTorch (may slow down performance slightly)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# set seed
set_seed(3407)

# Step 1: read image
img = read_image("SVS_1_SC4_LS28_to_LS29_0000_0_39_768_1024.jpg")
input_tensor = normalize(resize(img, (224, 224)) / 255., [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

# Step 2: load model
model = resnet18(pretrained=True)
# model.fc = nn.Linear(512, 2)
# model.load_state_dict(torch.load('checkpoints/output/epoch=99-val_loss=0.00.ckpt'))
model.eval()  # Set the model to evaluation mode after loading
cam_extractor = GradCAM(model, 'layer4')
out = model(input_tensor.unsqueeze(0))

# Step 3: extract activation map
activation_map = cam_extractor(out.squeeze(0).argmax().item(), out)

# Step 4:  convert to numpy array
heatmap = activation_map[0].squeeze(0).detach().numpy()

# smooth the heatmap values to avoid too dispersed sampling probabilities, enhance the sampling probability of the high temperature area
heatmap = np.clip(heatmap, 0, None)  # remove negative values
heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())  # normalize to [0, 1]

# Step 5: resize heatmap to match the original image size
heatmap_resized = cv.resize(heatmap, (img.shape[2], img.shape[1]))

# Step 6: sampling logic
def sample_points(heatmap, num_points=10, weight_exponent=1):
    height, width = heatmap.shape
    flat_heatmap = heatmap.flatten()

    flat_heatmap = np.power(flat_heatmap, weight_exponent)

    probabilities = flat_heatmap / flat_heatmap.sum() 
    indices = np.arange(len(flat_heatmap))

    sampled_indices = np.random.choice(indices, size=num_points, replace=False, p=probabilities)

    sampled_coords = [(index % width, index // width) for index in sampled_indices]

    return sampled_coords

# Step 7: get sampled points
sampled_points = sample_points(heatmap_resized, num_points=10, weight_exponent=1)

# Step 8:  load SAM model
sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

# Step 9: convert image to PIL format
image = to_pil_image(img)
predictor.set_image(np.array(image))

# Step 11: convert sampled points to the format required by the SAM model
input_points = np.array(sampled_points)
input_labels = np.ones(input_points.shape[0])

# Step 10: run SAM model
masks, scores, _ = predictor.predict(
    point_coords=input_points, 
    point_labels=input_labels,
    multimask_output=False
)


# Step 12:  determine whether the foreground or background is larger
foreground_count = np.sum(masks)
background_count = np.size(masks) - foreground_count
if foreground_count > background_count:
    masks = np.logical_not(masks)

# Step 12: post-processing of the mask
def post_process_mask(mask):
    # Convert mask to uint8 for OpenCV operations
    mask = (mask * 255).astype(np.uint8)

    kernel = np.ones((3, 3), np.uint8)  
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)

    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    min_area = 100
    max_area = 15000

    filtered_image = np.zeros_like(mask)

    for contour in contours:
        area = cv.contourArea(contour)
        if min_area < area < max_area:
            cv.drawContours(filtered_image, [contour], -1, 255, thickness=cv.FILLED)

    return filtered_image

processed_mask = post_process_mask(masks[0])

    
# Step 12: visualize the results
def show_points(coords, labels, ax, marker_size=150):
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    
plt.figure(figsize=(20, 50))

# original
plt.subplot(1, 5, 1)
plt.imshow(image)
plt.axis('off')
plt.title('Original')

# CAM overlay
result = overlay_mask(to_pil_image(img), to_pil_image(activation_map[0].squeeze(0), mode='F'), alpha=0.5)
plt.subplot(1, 5, 2)
plt.imshow(result)
plt.title('CAM Overlay')
plt.axis('off')

# show sampled points
plt.subplot(1, 5, 3)
plt.imshow(image)
show_points(input_points, input_labels, plt.gca())
plt.axis('off')
plt.title('Pointed')    


# show SAM mask
plt.subplot(1, 5, 4)
plt.imshow(image)
plt.imshow(masks[0], cmap='jet', alpha=0.5)  # 使用半透明的掩码叠加到原图上
plt.axis('off')
plt.title('SAM')

# show post-processed mask
plt.subplot(1, 5, 5)
plt.imshow(image)
plt.imshow(processed_mask, cmap='jet', alpha=0.5)  # 叠加处理后的掩码
plt.axis('off')
plt.title('Post-Processed')
plt.show()
